# Moody & Wu (1997) — Optimización en línea con razón de Sharpe diferencial

Baseline educativo: estrategia parametrizada \(s_t = 	anh(w^	op x_t)\) y **entrenamiento en línea** para maximizar una aproximación de la **diferential Sharpe**.


Incluye:
1) Datos sintéticos con tendencia AR + ruido.
2) Señal paramétrica y PnL.
3) Estimación recursiva de media/varianza.
4) Paso de gradiente aproximado sobre un proxy diferenciable del Sharpe.


## 1) Imports

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
rng = np.random.default_rng(7)


## 2) Serie sintética tipo AR(1) en rendimientos

In [ ]:

def synthetic_returns(n=4000, ar=0.05, sigma=0.01, seed=7):
    rng = np.random.default_rng(seed)
    r = np.zeros(n, dtype=float)
    for t in range(1, n):
        r[t] = ar * r[t-1] + rng.normal(0, sigma)
    return r

r = synthetic_returns()
plt.figure()
plt.plot(r[:500])
plt.title('Rendimientos sintéticos (primeros 500)')
plt.show()


## 3) Señal paramétrica y PnL

In [ ]:

def features(r, k=5):
    # k rezagos como features
    X = []
    for t in range(k, len(r)):
        X.append(r[t-k:t][::-1])
    return np.array(X), r[k:]

def pnl_from_signal(signal, ret, cost=0.0002):
    # PnL_t = signal_{t-1} * ret_t - cost * |Δsignal_t|
    ds = np.diff(signal, prepend=0.0)
    pnl = signal * ret - cost * np.abs(ds)
    return pnl

k = 10
X, y = features(r, k=k)
# inicialización
w = np.zeros(k)
signal = np.tanh(X @ w)
pnl = pnl_from_signal(signal, y)


## 4) Estimación recursiva de media/varianza y entrenamiento en línea

In [ ]:

def online_train_sharpe(X, y, epochs=1, lr=0.05, beta=0.01, ema=0.01, cost=0.0002):
    # beta: pequeño término para estabilizar varianza
    # ema: factor para EMA de media y varianza
    w = np.zeros(X.shape[1])
    m = 0.0  # media EMA
    v = 1e-4  # var EMA inicial
    history = {"SR":[]}
    for ep in range(epochs):
        for t in range(len(y)):
            x = X[t]
            s = np.tanh(x @ w)
            # forward
            if t == 0:
                ds = s
            else:
                s_prev = np.tanh(X[t-1] @ w)
                ds = s - s_prev
            pnl = s * y[t] - cost * abs(ds)
            # actualizar momentos
            m = (1-ema)*m + ema*pnl
            v = (1-ema)*v + ema*(pnl - m)**2
            # proxy diferenciable para Sharpe ~ m / sqrt(v + beta)
            sr_proxy = m / np.sqrt(v + beta)
            history["SR"].append(sr_proxy)
            # gradiente aproximado por REINFORCE-like surrogate
            # d pnl / d w ≈ (1 - s**2) * x * y[t]  - cost * sign(ds) * (1 - s**2) * x
            ds_sign = np.tanh(1000*ds)  # aprox suave de sign
            dpnl_dw = (1 - s**2) * x * y[t] - cost * ds_sign * (1 - s**2) * x
            # d sr / d pnl ≈ ∂(m / sqrt(v+β)) / ∂pnl usando aproximación EMA -> tratamos pnl como influir m,v localmente
            d_sr_dm = 1/np.sqrt(v + beta)
            d_sr_dv = -0.5*m*(v+beta)**(-1.5)
            # aproximamos ∂m/∂pnl ≈ ema, ∂v/∂pnl ≈ 2*ema*(pnl - m)
            d_sr_dpnl = d_sr_dm*ema + d_sr_dv*(2*ema*(pnl - m))
            grad = d_sr_dpnl * dpnl_dw
            w += lr * grad
    return w, history

w_tr, hist = online_train_sharpe(X, y, epochs=3, lr=0.03, ema=0.02, cost=0.0003)
signal_tr = np.tanh(X @ w_tr)
pnl_tr = pnl_from_signal(signal_tr, y)
# SR ex-post simple
SR = pnl_tr.mean() / (pnl_tr.std()+1e-9)
print("Sharpe ex-post:", SR)

plt.figure()
plt.plot(np.cumsum(pnl_tr))
plt.title('Equity curve (cumulative PnL)')
plt.show()


## 5) Notas
- Proxy del gradiente del Sharpe diferencial, enfoque didáctico.
- Extender con RTRL/GRU si se desea memoria explícita.
- Evaluar robustez con *walk-forward* y *rolling windows*. 